In [1]:
import re
import string
import nltk
import spacy
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy


pd.set_option('display.max_colwidth', 200)

In [2]:

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Load the CSV file into a Pandas DataFrame
our_data = pd.read_csv('../goetschm/maic/asd-hackathon/ASD-Hackathon/big_data_energy/our_data.csv', encoding = "utf-8")
input_text = pd.read_csv('../goetschm/maic/asd-hackathon/ASD-Hackathon/big_data_energy/ns_io.csv', encoding = "utf-8")


In [3]:
# function to preprocess speech
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing input
df = pd.DataFrame()
df['clean_in'] = input_text['Input'].apply(clean)

# split sentences
def sentences(text):
    # split sentences and questions
    text = re.split('[.?]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

# sentences
df['sent'] = df['clean_in'].apply(sentences)

row_list = []

for i in range(len(df)):
    for sent in df.loc[i,'sent']:
        dict1 = {'sent':sent}
        row_list.append(dict1)
    
df2 = pd.DataFrame(row_list)


In [4]:
def prog_sent(text):
    
    patterns = [r'\b(?i)'+'therapy'+r'\b',
               r'\b(?i)'+'program'+r'\b',
               r'\b(?i)'+'treatment'+r'\b',
               r'\b(?i)'+'evaluation'+r'\b',
               r'\b(?i)'+'referral'+r'\b',
               r'\b(?i)'+'ABA'+r'\b',
               r'\b(?i)'+'group'+r'\b']

    output = []
    flag = 0
    for pat in patterns:
        if re.search(pat, text) != None:
            flag = 1
            break
    return flag 

# apply function
df2['therapy'] = df2['sent'].apply(prog_sent)

/tmp/ipykernel_165331/3862973895.py:14: DeprecationWarning: Flags not at the start of the expression '\\b(?i)therapy\\b' but at position 2
  if re.search(pat, text) != None:
/tmp/ipykernel_165331/3862973895.py:14: DeprecationWarning: Flags not at the start of the expression '\\b(?i)program\\b' but at position 2
  if re.search(pat, text) != None:
/tmp/ipykernel_165331/3862973895.py:14: DeprecationWarning: Flags not at the start of the expression '\\b(?i)treatment\\b' but at position 2
  if re.search(pat, text) != None:
/tmp/ipykernel_165331/3862973895.py:14: DeprecationWarning: Flags not at the start of the expression '\\b(?i)evaluation\\b' but at position 2
  if re.search(pat, text) != None:
/tmp/ipykernel_165331/3862973895.py:14: DeprecationWarning: Flags not at the start of the expression '\\b(?i)referral\\b' but at position 2
  if re.search(pat, text) != None:
/tmp/ipykernel_165331/3862973895.py:14: DeprecationWarning: Flags not at the start of the expression '\\b(?i)ABA\\b' but at 

In [5]:
# df['tokenz']= df['sent'].apply(lambda x: nlp(x))

# to extract therapy using pattern matching
def all_schemes(text,check):
    
    keyword = []
    
    doc = nlp(text)
    
    # initiatives
    phrases = ['therapy', 'evaluation']
    
   
    patterns = list(nlp.tokenizer.pipe(phrases))

    if check == 0:
        # return blank list
        return schemes

    # Matcher class object 
    matcher = PhraseMatcher(nlp.vocab) 
    # matcher.add("matching", pattern) 
    matcher.add('KeywordMatcher', patterns)

    matches = matcher(doc)
    print(matches)

    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id] 
        span = doc[start:end]  
        print(string_id, start, end, span.text)
        
    return keyword

# apply function
df2['therapy1'] = df2.apply(lambda x:all_schemes(x.sent,x.therapy),axis=1)
df2

[(16303965782349502793, 7, 8)]
KeywordMatcher 7 8 evaluation
[]
[(16303965782349502793, 16, 17)]
KeywordMatcher 16 17 evaluation
[]
[]
[(16303965782349502793, 15, 16)]
KeywordMatcher 15 16 therapy
[(16303965782349502793, 20, 21)]
KeywordMatcher 20 21 therapy


,sent,therapy,therapy1
0,My 3 year old had a high screening for autism,0,[]
1,Where can I go for an evaluation,1,[]
2,The referral given by the pediatrician is a long wait and I want other options,1,[]
3,,0,[]
4,A parent of a 3 year old is wondering where they can go to get an evaluation for their child who had a high screening for autism,1,[]
5,The referral given by the pediatrician is a long wait and they want other options,1,[]
6,,0,[]
7,A parent of a child recently diagnosed with autism is trying to figure out what treatment options are available to them and where to go for services,1,[]
8,A provider is looking for a list of treatment providers for ABA/autism behavior therapy who treat children age 10 years old,1,[]
9,A caregiver has a child diagnosed with autism and they are currently on a waitlist for ABA/autism behavior therapy,1,[]


In [6]:
sample = nlp(df2['sent'][9])
displacy.render(sample, style='dep',jupyter=True)
displacy.render(sample, style="ent")
for token in sample:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


A a DET DT det X True True
caregiver caregiver NOUN NN nsubj xxxx True False
has have VERB VBZ ROOT xxx True True
a a DET DT det x True True
child child NOUN NN dobj xxxx True False
diagnosed diagnose VERB VBN acl xxxx True False
with with ADP IN prep xxxx True True
autism autism NOUN NN pobj xxxx True False
and and CCONJ CC cc xxx True True
they they PRON PRP nsubj xxxx True True
are be AUX VBP conj xxx True True
currently currently ADV RB advmod xxxx True False
on on ADP IN prep xx True True
a a DET DT det x True True
waitlist waitlist NOUN NN pobj xxxx True False
for for ADP IN prep xxx True True
ABA ABA PROPN NNP nmod XXX True False
/ / SYM SYM punct / False False
autism autism NOUN NN compound xxxx True False
behavior behavior NOUN NN compound xxxx True False
therapy therapy NOUN NN pobj xxxx True False


In [7]:
doc = nlp("My 3-year-old had a high screening for autism. Where can I go for an evaluation? The referral given by the pediatrician is a long wait and I want other options.")
doc

My 3-year-old had a high screening for autism. Where can I go for an evaluation? The referral given by the pediatrician is a long wait and I want other options.

In [8]:

# print token, dependency, POS tag 
for tok in nlp(df2['sent'][0]): 
  print(tok.text, "-->",tok.dep_,"-->", tok.pos_)


My --> poss --> PRON
3 --> nummod --> NUM
year --> npadvmod --> NOUN
old --> nsubj --> ADJ
had --> ROOT --> VERB
a --> det --> DET
high --> amod --> ADJ
screening --> dobj --> NOUN
for --> prep --> ADP
autism --> pobj --> NOUN
